# LGBM/XGBoosting for the traffic flow data

https://www.kaggle.com/robikscube/tutorial-time-series-forecasting-with-xgboost

Importing libraries

In [ ]:
# general routine set up:

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# get wd and change the wd plus import libraries
import os
os.getcwd()
#os.chdir("/Users/Manu/Dropbox/CBS MSc Thesis Research Folder/DATA & Code/Model Specific Notebooks")


# standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from math import sqrt
import pickle

# ML
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
import tensorflow.keras.backend as K
from lightgbm import LGBMRegressor

Loading the data

In [ ]:
# this allows for accessing files stored in your google drive using the path "/gdrive/"
# mounting google drive locally:

from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
#loading the hourly traffic data (1 year of data; June 2018 to June 2019)
filename = "/gdrive/My Drive/Colab Notebooks/taxi_series_H"
infile = open(filename,'rb')
taxidemand_ts = pickle.load(infile)
infile.close()

In [ ]:
taxidemand_ts.shape

(8760,)

Preprocessing: already done


Time Series data must be re-framed as a supervised learning dataset before we can start using machine learning algorithms.  
There is no concept of input and output features in time series. Instead, we must choose the variable to be predicted and use feature engineering to construct all of the inputs that will be used to make predictions for future time steps.

## Feature Engineering for Times Series

[feature_eng_ts.png](attachment:feature_eng_ts.png)

In this tutorial, we will look at three classes of features that we can create from our time series dataset:

    Date Time Features: these are components of the time step itself for each observation.
    Lag Features: these are values at prior time steps.
    Window Features: these are a summary of values over a fixed window of prior time steps.


Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

The goal of feature engineering is to provide strong and ideally simple relationships between new input features and the output feature for the supervised learning algorithm to model.

**Preprocessing for Multioutput Forecasting**

### Preprocessing (For multi output model)
For supervised machine learning methods to work on time series it is necessary to do a certain amount of preprocessing. This includes feature generation such as embedding(lags). 

In [ ]:
## Remove last 24 hours as well as the part of test set not being used for main results
taxidemand_ts = taxidemand_ts.iloc[0:-192]
taxidemand_ts.shape

(8568,)

In [ ]:
# export the preprocessed data for plotting in R
taxidemand_ts.to_csv('/gdrive/My Drive/Colab Notebooks/traffic_1yr_plot.csv', index=True)

In [ ]:
## Set paramaters
data = taxidemand_ts.copy()
input_lags = 60 ## number of lags to be used for input. should be 2,5 times the seasonal period 
output_lags = 24 ## number of future oberservations to be forecasted
n_test = 24 ## size of test set 

In [ ]:
## Split data in train and test set
train = data[0:-n_test]
test = data[-n_test:]
print(train.shape)
print(test.shape)

(8544,)
(24,)


In [ ]:
## Create dataframe with the correct dimensions. Each row represents the past 54 observations and the 24 future observations
df = pd.DataFrame()
n_train = len(train)

## create inputs lags
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = train.shift(i)

## create output lags
for j in range(0,output_lags,1):
    df['t+' + str(j)] = train.shift(-j)

## remove the first input_lags rows and last output_lags rows   
df = df[input_lags:(n_train-output_lags+1)]  

print(df.shape)

(8461, 84)


In [ ]:
## Split train into features X and targets Y
X_train = df.iloc[:,:input_lags] 
y_train = df.iloc[:,input_lags:]

## To create the test features X_test we cannot use any of the test set, since that includes the data held out
# for testing. We therefor use the last input_lags number of observations in the training set. These are however
# split between the targets and features and requires a combination of the two. (output_lags) from the targets 
# and (input_lags - output_lags) from the features
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:] ## First get the last (input_lags - output_lags) from the features 
X_test = X_test.append(y_train.iloc[len(y_train) - 1,:]) ## Second, add the last (output_lags) from the targets

In [ ]:
## Remodel as numpy arrays and reshape
X_train_multi = X_train.values ## should be (n - input_lags - outputlags - n_test + 1) x (input_lags)
y_train_multi = y_train.values ## should be (n - input_lags - outputlags - n_test + 1) x (output_lags)
X_test_multi = X_test.values.reshape(1,input_lags) ## should be (1) x (input_lags) 
y_test_multi = test.values.reshape(1,n_test) ## should be (1) x (n_test)

print("X_train_multi: " + "type: " + str(type(X_train_multi)) + "\tshape: " + str(X_train_multi.shape))
print("y_train_multi: " + "type: " + str(type(y_train_multi)) + "\tshape: " + str(y_train_multi.shape))
print("X_test_multi: " + "type: " + str(type(X_test_multi)) + "\tshape: " + str(X_test_multi.shape))
print("y_test_multi: " + "type: " + str(type(y_test_multi)) + "\tshape: " + str(y_test_multi.shape))

X_train_multi: type: <class 'numpy.ndarray'>	shape: (8461, 60)
y_train_multi: type: <class 'numpy.ndarray'>	shape: (8461, 24)
X_test_multi: type: <class 'numpy.ndarray'>	shape: (1, 60)
y_test_multi: type: <class 'numpy.ndarray'>	shape: (1, 24)


$\textbf{General motivation for XG boosting}:$
One advantages of the XGBoosting algorithm is that it comes with the built in option to include regularization which is why it is also called a regularized boosting sometimes.
It also comes with a built in cross validation option and it provides plenty of flexiblities for parameter tuning. XGboosting makes splits up to the specified "max_depth" parameter and then prunes the trees backwards.
The iterative nature of this algorithm is also an advantage: one can build a new model based on the specification of the previous iteration, which has been implemented during the fine tuning of parameters in this case.

$\textbf{General setup of the algorithm}:$ As far as general parameters go, the booster "gbtree" has been used here, ie. I have been using tree based models in each iteration instead of a linear model, which is rarely used. I have started out with a number of estimators of 1,000 and then I have fine tuned the following parameters after try 2: max_depth and min_child_weight (gsearch1), reg_alpha (gsearch2), gamma (gsearch3), subsample and colsample_bytree (gsearch4), reg_lambda (gsearch5).  
I have used a feature importance plot of the XGBClassfier model to select the most important features to include. I have included the top 10 features out of the 24 features provided from try 3 onwards.
The fine tuned parameters have the following influence on the xg boosting algorithm:  
  
$\textit{max_depth:}$ specifies the max depth of a tree and can be used to control overfitting as higher depth will allow the model to learn relations very specific to a particular sample   
$\textit{min_child_weight:}$ this sets the minimum sum of weights of all observations required in a child. Higher values prevent the model from learning too specific relations.  
$\textit{reg_alpha:}$ L1 regularization term. Can be used in case of high dimensionality to make the algorithm run faster. Can be a solution to overfitting in case of a relatively small dataset.  
$\textit{gamma:}$ sets the minimum loss function required to make a split.  
$\textit{subsample:}$ sets the fraction of observations to be random samples of each tree. lower values prevent overfitting but small too small values might lead to underfitting.  
$\textit{colsample_bytree:}$ fraction of columns to be random samples of each tree.  
$\textit{reg_lambda:}$ L2 regularization term. can be a solution to overfitting in case of a relatively small dataset. Can be explored to reduce overfitting.

## XG (Extreme Gradient) boosting 

## Gridsearch CV

**Blogpost for XGB Hyperparameter tuning:**

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

**Full grid:**

xgbreg_cv = GridSearchCV(estimator = xgb_model, param_grid = {"colsample_bytree":[0.6,0.8,1.0], 'subsample':[0.8,1.0],"min_child_weight":[1.0,1.2], 'learning_rate': [0.1,0.05,0.01]
, 'max_depth': [5,8,10], 'n_estimators': [500,1000], 'lambda': [0.7,0.8,1], 'alpha':[0.7,0.8,1.0]}, verbose=1, cv = inner_loop, n_jobs=3)

**Gridsearch CV for the Multioutput XGB model**

In [ ]:
inner_splits = 3
inner_loop = TimeSeriesSplit(n_splits = inner_splits).split(X_train_multi,y_train_multi)

# lambda: l2(ridge) regularization term on weights, alpha: l1(lasso) regularization term on weights
# njobs: parallel processing -> speeds up the process a lot (assumption: quad core CPU -> n_jobs = 4)
# nthread set to -1 uses all the cores available in the system
xgb_model_multi = MultiOutputRegressor(xgb.XGBRegressor(objective = "reg:squarederror", booster='gbtree', nthread=-1))
xgbreg_cv_multi = GridSearchCV(estimator = xgb_model_multi, param_grid = {"estimator__colsample_bytree":[0.8,1.0], 'estimator__subsample':[0.8],"estimator__min_child_weight":[1,3,5], 'estimator__learning_rate': [0.1,0.05,0.01]
, 'estimator__max_depth': [5,8,10], 'estimator__n_estimators': [500,1000], 'estimator__lambda': [0.7,1.0], 'estimator__alpha':[0.7,1.0]}, verbose=1, cv = inner_loop, n_jobs=4)

Slightly reduced grid

In [ ]:
inner_splits = 3
inner_loop = TimeSeriesSplit(n_splits = inner_splits).split(X_train_multi,y_train_multi)

# lambda: l2(ridge) regularization term on weights, alpha: l1(lasso) regularization term on weights
# njobs: parallel processing -> speeds up the process a lot (assumption: quad core CPU -> n_jobs = 4)
# nthread set to -1 uses all the cores available in the system
xgb_model_multi = MultiOutputRegressor(xgb.XGBRegressor(objective = "reg:squarederror", booster='gbtree', nthread=-1))
xgbreg_cv_multi = GridSearchCV(estimator = xgb_model_multi, param_grid = {"estimator__colsample_bytree":[0.8,1.0], 'estimator__subsample':[0.8],"estimator__min_child_weight":[3,5], 'estimator__learning_rate': [0.1,0.05,0.01]
, 'estimator__max_depth': [5,8], 'estimator__n_estimators': [500], 'estimator__lambda': [0.7,1.0], 'estimator__alpha':[0.7,1.0]}, verbose=1, cv = inner_loop, n_jobs=4)

In [ ]:
xgbreg_cv_multi.fit(X_train_multi,y_train_multi)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 13.3min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 66.3min
[Parallel(n_jobs=4)]: Done 288 out of 288 | elapsed: 102.1min finished


GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7fa907727150>,
             error_score=nan,
             estimator=MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5,
                                                                   booster='gbtree',
                                                                   colsample_bylevel=1,
                                                                   colsample_bynode=1,
                                                                   colsample_bytree=1,
                                                                   gamma=0,
                                                                   importance_type='gain',
                                                                   learning_rate=0.1,
                                                                   max_delta_step=0,
                                                                   max_depth=3,
                                              

In [ ]:
# obtaining the best hyperparameter values from the gridsearch cv
xgbreg_cv_multi.best_params_

{'estimator__alpha': 0.7,
 'estimator__colsample_bytree': 0.8,
 'estimator__lambda': 0.7,
 'estimator__learning_rate': 0.05,
 'estimator__max_depth': 5,
 'estimator__min_child_weight': 5,
 'estimator__n_estimators': 500,
 'estimator__subsample': 0.8}

#LightGBM (Gradient Boosting)



https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

lightgbm parameters:

https://lightgbm.readthedocs.io/en/latest/Parameters.html



Gridsearch CV 


New Grid tailor made for LGBM

In [ ]:
# LGBM specific new grid

inner_splits = 3
inner_loop = TimeSeriesSplit(n_splits = inner_splits).split(X_train_multi,y_train_multi)

# lambda: l2(ridge) regularization term on weights, alpha: l1(lasso) regularization term on weights 
# njobs: parallel processing -> speeds up the process a lot (assumption: quad core CPU -> n_jobs = 4)
# nthread set to -1 uses all the cores available in the system

# num_leaves: important parameter that controls the complexity of the tree model
# min_data_in_leaf: another important parameter to avoid overfitting of the leaf-wise tree algorithm. small -> overfitting
# This value is affected by the number of training sets and num_leaves. 
# Setting this parameter larger can avoid growing too deep trees, but also avoid underfittinng

# setting up the model with baseline parameters
lgbm_model_multi_new = MultiOutputRegressor(LGBMRegressor(boosting_type = "gbdt", objective = "root_mean_squared_error", n_jobs=-1))




In [ ]:
lgbm_cv_multi_new = GridSearchCV(estimator = lgbm_model_multi_new, param_grid = {"estimator__num_leaves":[30,40], "estimator__min_child_samples":[20,30], "estimator__colsample_bytree":[0.8,1.0],'estimator__subsample':[1.0],"estimator__min_child_weight":[0.01,3], 'estimator__learning_rate': [0.1,0.01],'estimator__max_depth': [5,8], 'estimator__n_estimators': [250,500], 'estimator__reg_lambda': [0.0,0.7], 'estimator__reg_alpha':[0.0,0.7]}, verbose=1, cv = inner_loop, n_jobs=-1, scoring="neg_root_mean_squared_error")

In [ ]:
lgbm_cv_multi_new.fit(X_train_multi,y_train_multi) 

Fitting 3 folds for each of 512 candidates, totalling 1536 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 38.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 123.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 275.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 439.9min
[Parallel(n_jobs=-1)]: Done 1536 out of 1536 | elapsed: 605.1min finished


GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7f08a603f888>,
             error_score=nan,
             estimator=MultiOutputRegressor(estimator=LGBMRegressor(boosting_type='gbdt',
                                                                    class_weight=None,
                                                                    colsample_bytree=1.0,
                                                                    importance_type='split',
                                                                    learning_rate=0.1,
                                                                    max_depth=-1,
                                                                    min_child_samples=20,
                                                                    min_child_weight=0.001,
                                                                    min_split_gain=0.0,
                                                                    n_estimators=100,
         

In [ ]:
# obtaining the best hyperparameter values from the gridsearch cv
lgbm_cv_multi_new.best_params_

{'estimator__colsample_bytree': 0.8,
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': 8,
 'estimator__min_child_samples': 30,
 'estimator__min_child_weight': 0.01,
 'estimator__n_estimators': 250,
 'estimator__num_leaves': 30,
 'estimator__reg_alpha': 0.0,
 'estimator__reg_lambda': 0.7,
 'estimator__subsample': 1.0}

In [ ]:
lgbm_cv_multi_new.best_score_

0.9023750407104661

## Fitting the optimal model

# XGBoosting Optimal model

In [ ]:
# learning rate is the step size shrinkage used to prevent overfitting. Range is [0,1]
# lambda: default = 1,  L2 regularization term on weights (analogous to Ridge regression)
# this used to handle the regularization part of XGBoost. Though many data scientists don’t use it often, 
# it should be explored to reduce overfitting.
# max_depth is also used to control overfitting as higher depth will allow model to learn relations very specific to a particular sample

# alpha: L1 regularization on leaf weights. A large value leads to more regularization
# In order to build more robust models, it is common to do a k-fold cross validation where 
# all the entries in the original training dataset are used for both training as well as validation
# XGBoost supports k-fold cross validation via the cv() method. All you have to do is specify the nfolds parameter, 
# which is the number of cross validation sets you want to build.


xgb_reg_opt=xgb.XGBRegressor(random_state=1,learning_rate=0.05, objective = "reg:squarederror", n_estimators =500, colsample_bytree = 0.8, max_depth = 5, subsample= 0.8, min_child_weight =5, reg_lambda = 0.7, alpha= 0.7, eval_metric = "rmse")

## Fitting a single output regressor xgb model
#xgb_reg_opt.fit(X_train, y_train)

# Fitting a Multioutputregressor xgb model to obtain multioutput forecasts

multioutputregressor_fit = MultiOutputRegressor(xgb_reg_opt).fit(X_train_multi, y_train_multi)


In [ ]:
# (lag) feature importance scores
xgb_reg_opt.feature_importances_

array([0.00218971, 0.00434436, 0.00916981, 0.00202504, 0.00149078,
       0.00168853, 0.00073392, 0.0034906 , 0.00096166, 0.00130496,
       0.00144312, 0.00224894, 0.0008032 , 0.00142685, 0.00070412,
       0.00225331, 0.00078325, 0.00066303, 0.00090281, 0.00085732,
       0.00087486, 0.00072116, 0.00092707, 0.00064558, 0.00161666,
       0.00651186, 0.002436  , 0.00116427, 0.00075428, 0.00072861,
       0.05196435, 0.05025709, 0.03885129, 0.01461791, 0.00254124,
       0.00161706, 0.00094993, 0.00071457, 0.01119858, 0.0036468 ,
       0.00103879, 0.00112749, 0.0008306 , 0.00107581, 0.00689208,
       0.00202745, 0.0006693 , 0.0030678 , 0.07589586, 0.11545025,
       0.00691677, 0.00203558, 0.00195733, 0.54879045], dtype=float32)

#LightGBM optimal model


In [ ]:
# best model
lgbm_reg_opt_new = LGBMRegressor(random_state=1,learning_rate=0.1, objective = "root_mean_squared_error", n_estimators =500, colsample_bytree = 0.8, max_depth = 8, subsample= 1.0, num_leaves=30, min_child_samples=20, min_child_weight =3, reg_lambda = 0.7, reg_alpha= 0.0, eval_metric = "rmse",boosting_type="gbdt")

lgbm_moreg_fit_new = MultiOutputRegressor(lgbm_reg_opt_new).fit(X_train_multi, y_train_multi)

#Quantile Regression loss function for Prediction intervals

Way 1 (lightgbm)


In [ ]:
#### 95% Prediction Intervals - Upper bound ####
# upper bound 
alpha = 0.975


# Gridsearch CV for alpha = 0.975 and a quantile loss function

inner_splits = 3
inner_loop = TimeSeriesSplit(n_splits = inner_splits).split(X_train_multi,y_train_multi)

# lambda: l2(ridge) regularization term on weights, alpha: l1(lasso) regularization term on weights
# njobs: parallel processing -> speeds up the process a lot (assumption: quad core CPU -> n_jobs = 4)
# nthread set to -1 uses all the cores available in the system
lgbm_multi_pi_upper = MultiOutputRegressor(LGBMRegressor(boosting_type = "gbdt", objective = "quantile", n_jobs=-1, alpha = alpha))
lgbm_multi_cv_upper = GridSearchCV(estimator = lgbm_multi_pi_upper, param_grid = {"estimator__colsample_bytree":[0.8,1.0],'estimator__subsample':[1.0],"estimator__min_child_weight":[3,5], 'estimator__learning_rate': [0.1,0.01,0.001]
, 'estimator__max_depth': [5,8], 'estimator__n_estimators': [500], 'estimator__reg_lambda': [0.7,1.0], 'estimator__reg_alpha':[0.7,1.0]}, verbose=1, cv = inner_loop, n_jobs=-1)





In [ ]:
lgbm_multi_cv_upper.fit(X_train_multi,y_train_multi)

In [ ]:
# obtaining the best hyperparameter values from the gridsearch cv (upper bound)
lgbm_multi_cv_upper.best_params_

**Optimal parameters upper bound GSCV:**

{'estimator__colsample_bytree': 0.8,
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': 5,
 'estimator__min_child_weight': 3,
 'estimator__n_estimators': 500,
 'estimator__reg_alpha': 0.7,
 'estimator__reg_lambda': 1.0,
 'estimator__subsample': 0.8}
 

In [ ]:
# Fitting the optimal PI model for the upper PI bound (alpha = 0.975 & quantile loss)

alpha = 0.975

lgbm_reg_pi=LGBMRegressor(random_state=1,learning_rate=0.1, objective = "quantile", n_estimators =500, colsample_bytree = 0.8, max_depth = 5, subsample= 1.0, min_child_weight =3, reg_lambda = 1.0, reg_alpha= 0.7, alpha=alpha, eval_metric = "rmse")


lgbm_reg_fit_upper = MultiOutputRegressor(lgbm_reg_pi).fit(X_train_multi, y_train_multi)

# Make the prediction on the meshed x-axis
y_upper = lgbm_reg_fit_upper.predict(X_test_multi)

In [ ]:
#### 95% Prediction Intervals - Lower bound ####

alpha = 0.025

# Gridsearch CV for alpha = 0.025 and a quantile loss function

inner_splits = 3
inner_loop = TimeSeriesSplit(n_splits = inner_splits).split(X_train_multi,y_train_multi)

# lambda: l2(ridge) regularization term on weights, alpha: l1(lasso) regularization term on weights
# njobs: parallel processing -> speeds up the process a lot (assumption: quad core CPU -> n_jobs = 4)
# nthread set to -1 uses all the cores available in the system
lgbm_multi_pi_lower = MultiOutputRegressor(LGBMRegressor(boosting_type = "gbdt", objective = "quantile", n_jobs=-1, alpha = alpha))
lgbm_multi_cv_lower = GridSearchCV(estimator = lgbm_multi_pi_lower, param_grid = {"estimator__colsample_bytree":[0.8,1.0],'estimator__subsample':[1.0],"estimator__min_child_weight":[3,5], 'estimator__learning_rate': [0.1,0.01,0.001]
, 'estimator__max_depth': [5,8], 'estimator__n_estimators': [500], 'estimator__reg_lambda': [0.7,1.0], 'estimator__reg_alpha':[0.7,1.0]}, verbose=1, cv = inner_loop, n_jobs=-1)




In [ ]:
lgbm_multi_cv_lower.fit(X_train_multi,y_train_multi)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 14.7min
[Parallel(n_jobs=4)]: Done 288 out of 288 | elapsed: 22.9min finished


GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7fc9e695ed00>,
             error_score=nan,
             estimator=MultiOutputRegressor(estimator=LGBMRegressor(alpha=0.025,
                                                                    boosting_type='gbdt',
                                                                    class_weight=None,
                                                                    colsample_bytree=1.0,
                                                                    importance_type='split',
                                                                    learning_rate=0.1,
                                                                    max_depth=-1,
                                                                    min_child_samples=20,
                                                                    min_child_weight=0.001,
                                                                    min_split_gain=0.0,
              

In [ ]:
# obtaining the best hyperparameter values from the gridsearch cv (lower bound)
lgbm_multi_cv_lower.best_params_

{'estimator__colsample_bytree': 0.8,
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': 8,
 'estimator__min_child_weight': 3,
 'estimator__n_estimators': 500,
 'estimator__reg_alpha': 0.7,
 'estimator__reg_lambda': 1.0,
 'estimator__subsample': 0.8}

**Optimal hyperparams (lower bound) GSCV:**

{'estimator__colsample_bytree': 0.8,
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': 8,
 'estimator__min_child_weight': 3,
 'estimator__n_estimators': 500,
 'estimator__reg_alpha': 0.7,
 'estimator__reg_lambda': 1.0,
 'estimator__subsample': 0.8}
 

In [ ]:
# Fitting the optimal PI model for the lower PI bound (alpha = 0.025 & quantile loss)

alpha = 0.025

lgbm_reg_pi=LGBMRegressor(random_state=1,learning_rate=0.1, objective = "quantile", n_estimators =500, colsample_bytree = 0.8, max_depth = 8, subsample= 1.0, min_child_weight =3, reg_lambda = 1.0, reg_alpha= 0.7, alpha=alpha, eval_metric = "rmse")


lgbm_reg_fit_lower = MultiOutputRegressor(lgbm_reg_pi).fit(X_train_multi, y_train_multi)

# Make the prediction on the meshed x-axis
y_lower = lgbm_reg_fit_lower.predict(X_test_multi)

Way 2

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html

https://towardsdatascience.com/regression-prediction-intervals-with-xgboost-428e0a018b

In [ ]:
from functools import partial

In [ ]:
class XGBQuantile(xgb.XGBRegressor):
  def __init__(self,quant_alpha=0.95,quant_delta = 1.0,quant_thres=1.0,quant_var =1.0,base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0, learning_rate=0.1, eval_metric = "rmse", max_delta_step=0,max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
                n_jobs=1, nthread=None, objective='reg:linear', random_state=0,reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,silent=True, subsample=1):
    self.quant_alpha = quant_alpha
    self.quant_delta = quant_delta
    self.quant_thres = quant_thres
    self.quant_var = quant_var
    
    super().__init__(base_score=base_score, booster=booster, colsample_bylevel=colsample_bylevel,
       colsample_bytree=colsample_bytree, gamma=gamma, learning_rate=learning_rate, max_delta_step=max_delta_step,
       max_depth=max_depth, min_child_weight=min_child_weight, missing=missing, n_estimators=n_estimators,
       n_jobs= n_jobs, nthread=nthread, objective=objective, random_state=random_state,
       reg_alpha=reg_alpha, reg_lambda=reg_lambda, scale_pos_weight=scale_pos_weight, seed=seed,
       silent=silent, subsample=subsample)
    
    self.test = None
  
  def fit(self, X, y):
    super().set_params(objective=partial(XGBQuantile.quantile_loss,alpha = self.quant_alpha,delta = self.quant_delta,threshold = self.quant_thres,var = self.quant_var) )
    super().fit(X,y)
    return self
  
  def predict(self,X):
    return super().predict(X)
  
  def score(self, X, y):
    y_pred = super().predict(X)
    score = XGBQuantile.quantile_score(y, y_pred, self.quant_alpha)
    score = 1./score
    return score
      
  @staticmethod
  def quantile_loss(y_true,y_pred,alpha,delta,threshold,var):
    x = y_true - y_pred
    grad = (x<(alpha-1.0)*delta)*(1.0-alpha)-  ((x>=(alpha-1.0)*delta)& (x<alpha*delta) )*x/delta-alpha*(x>alpha*delta)
    hess = ((x>=(alpha-1.0)*delta)& (x<alpha*delta) )/delta 
 
    grad = (np.abs(x)<threshold )*grad - (np.abs(x)>=threshold )*(2*np.random.randint(2, size=len(y_true)) -1.0)*var
    hess = (np.abs(x)<threshold )*hess + (np.abs(x)>=threshold )
    return grad, hess
  
  @staticmethod
  def original_quantile_loss(y_true,y_pred,alpha,delta):
    x = y_true - y_pred
    grad = (x<(alpha-1.0)*delta)*(1.0-alpha)-((x>=(alpha-1.0)*delta)& (x<alpha*delta) )*x/delta-alpha*(x>alpha*delta)
    hess = ((x>=(alpha-1.0)*delta)& (x<alpha*delta) )/delta 
    return grad,hess

  
  @staticmethod
  def quantile_score(y_true, y_pred, alpha):
    score = XGBQuantile.quantile_cost(x=y_true-y_pred,alpha=alpha)
    score = np.sum(score)
    return score
  
  @staticmethod
  def quantile_cost(x, alpha):
    return (alpha-1.0)*x*(x<0)+alpha*x*(x>=0)
  
  @staticmethod
  def get_split_gain(gradient,hessian,l=1):
    split_gain = list()
    for i in range(gradient.shape[0]):
      split_gain.append(np.sum(gradient[:i])/(np.sum(hessian[:i])+l)+np.sum(gradient[i:])/(np.sum(hessian[i:])+l)-np.sum(gradient)/(np.sum(hessian)+l) )
    
    return np.array(split_gain)

## Obtaining predictions

**Multi-output model**

XGBoosting

In [ ]:
# running the retrained model on the test data
xgb_preds_multi = multioutputregressor_fit.predict(X_test_multi)
xgb_preds_multi.shape

(1, 24)

In [ ]:
# MSE
mse_xgbreg_multi = mean_squared_error(y_test_multi, xgb_preds_multi)
mse_xgbreg_multi

704699.0461237734

In [ ]:
# RMSE
sqrt(mse_xgbreg_multi)

839.4635466318794

In [ ]:
# MAE
mean_absolute_error(y_test_multi, xgb_preds_multi)

609.5590057373047

In [ ]:
# custom MAPE function
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true = y_test_multi
y_pred = xgb_preds_multi


In [ ]:
# MAPE
mean_absolute_percentage_error(y_true,y_pred)

7.614215276811718

LIGHTGBM

In [ ]:
lgbm_preds_new = lgbm_moreg_fit_new.predict(X_test_multi)

In [ ]:
# MSE
mse_lgbm_new = mean_squared_error(y_test_multi, lgbm_preds_new)
mse_lgbm_new

652952.8460385905

In [ ]:
# RMSE
sqrt(mse_lgbm_new)

808.0549771139279

In [ ]:
# MAE
mean_absolute_error(y_test_multi, lgbm_preds_new)

611.9593419973806

In [ ]:
# custom MAPE function
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true = y_test_multi
y_pred = lgbm_preds_new

In [ ]:
# MAPE
mean_absolute_percentage_error(y_true,y_pred)

9.39391830158383

**Exporting the Data as a CSV file:**


In [ ]:


# Creating a dataframe with GRU predictions and test data
LGBM_df = pd.DataFrame(np.transpose(lgbm_preds_multi))

# adding a column with the test data
LGBM_df["PI upper bound"] = np.transpose(y_upper)
LGBM_df["PI lower bound"] = np.transpose(y_lower)
LGBM_df["Test data"] = np.transpose(y_test_multi)


LGBM_df.columns = ['LGBM predictions', "PI upper bound", "PI lower bound", 'Test data']

In [ ]:
LGBM_df

,LGBM predictions,PI upper bound,PI lower bound,Test data
0,10775.684187,11989.102104,10052.530400,11284
1,8785.240102,10255.177836,6991.671354,9431
2,7417.886066,8736.246831,4155.632162,7524
3,4760.588988,6079.624763,2193.597025,5155
4,3079.153578,4226.169554,1730.670195,3075
5,2349.560298,4917.127980,2034.047477,1790
6,4080.604715,8291.035464,3362.204861,2571
7,6213.181292,11857.649223,4217.161542,3975
8,8325.297125,14743.700593,6441.890914,5945
9,9135.712462,13749.349079,8374.541383,8115


In [ ]:
# saving the dataframe as a CSV file
LGBM_df.to_csv('/gdrive/My Drive/Colab Notebooks/LGBM_preds.csv', index=False)